Analyze trip files converted from OMX matrices
1. Are trips sorted by departure time?
2. How does the temporal distribution look?
3. What are the distribution by mode, purpose, VoT?

In [1]:
import numpy as np
import pandas as pd
import h5py
import os
import gc
import time
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.float_format', lambda x: '%.3f' % x)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Execution Parameters

In [2]:
SWIFT_folder_local = r'C:\Projects\SWIFT\SWIFT_Workspace'
SWIFT_folder_L_drive = r'L:\DCS\Projects\_Legacy\60563434_SWIFT\400_Technical\SWIFT_Workspace'
SWIFT_folder = SWIFT_folder_L_drive
SWIFT_scenario_folder = os.path.join(SWIFT_folder, 'Scenarios')
SWIFT_common_data_folder = os.path.join(SWIFT_folder, 'CommonData\STM\STM_A\Shared_Inputs')
SCEN = 'S12_HG_HT_Infill_Pricing_N3'
flat_trip_file = os.path.join(SWIFT_scenario_folder, SCEN, r'STM\STM_A\01_DynusT\02_Demand\vehicle.csv')

In [3]:
matrix_folder = os.path.join(SWIFT_scenario_folder, SCEN, r'STM\STM_D\Outputs_SWIFT')
matrices_am = ["OD AM3HR Vehicles4DynusT EV", "OD AM3HR Vehicles4DynusT NonEV"]
matrices_md = ["OD MD6HR Vehicles4DynusT EV", "OD MD6HR Vehicles4DynusT NonEV"]
matrices_pm = ["OD PM4HR Vehicles4DynusT EV", "OD PM4HR Vehicles4DynusT NonEV"]
matrices_ov = ["OD OV8HR Vehicles4DynusT EV", "OD OV8HR Vehicles4DynusT NonEV"]
matrices = matrices_am + matrices_md + matrices_pm + matrices_ov
matrix_files = [os.path.join(matrix_folder, m+'.omx') for m in matrices]

In [4]:
vots = {
        "wi1da": 9.6, "wi2da": 15.04, "wi3da": 20.48, "wi4da": 27.52, "wi5da": 37.12,
        "wi12a2": 21.56, "wi3a2": 35.84, "wi45a2": 56.56,
        "wi12a3": 30.8, "wi3a3": 51.2, "wi45a3": 80.8,
        "nwi12da": 7.03, "nwi3da": 13.44, "nwi45da": 23.65,
        "nwi12a2": 12.3, "nwi3a2": 23.52, "nwi45a2": 41.39,
        "nwi12a3": 17.57, "nwi3a3": 33.6, "nwi45a3": 59.12,
        "Cargo": 64.0, "Serv": 40.0, "taxi_exta": 18.94, "TNC": 18.94
    }

map_purp = {
    1: "WK", 
    2: "NW", 
    3: "Taxi",
    4: "TNC",
    5: "Serv",
    6: "Cargo",
    
    10: "WK", 
    20: "NW", 
    30: "Taxi",
    40: "TNC",
    50: "Serv",
    60: "Cargo"
    
}

VTYPE_LABEL = {
        1:  'REG_SOV', 
        2:  'REG_HOV',
        3:  'REG_Serv',
        4:  'REG_Cargo',
        5:  'CAV_SOV',
        6:  'CAV_HOV',
        7:  'CAV_Serv',
        8:  'CAV_Cargo',
        9:  'TNC',
        
    }

In [5]:
diurnal_file = os.path.join(SWIFT_common_data_folder, 'Diurnal_Full.csv') 
print(f'Diurnal File Path: {diurnal_file}')

Diurnal File Path: L:\DCS\Projects\_Legacy\60563434_SWIFT\400_Technical\SWIFT_Workspace\CommonData\STM\STM_A\Shared_Inputs\Diurnal_Full.csv


### Diurnal Period Totals

In [6]:
df_diurnal = pd.read_csv(diurnal_file)
df_diurnal.head()

,HOUR,SHARE
0,0.000,0.000
1,0.002,0.000
2,0.003,0.000
3,0.005,0.000
4,0.007,0.000


In [7]:
def hour_to_period(hr):
    """
    Convert a float hour to text period "AM", "MD", "PM", "OV"
    """
    
    if 6 <= hr < 9:
        return 'AM'
    elif 9 <= hr < 15:
        return 'MD'
    elif 15 <= hr < 19:
        return 'PM'
    return 'OV'
    

In [8]:
df_diurnal['period'] = df_diurnal.HOUR.apply(hour_to_period)
diurnal_period_totals = df_diurnal.groupby('period', as_index=False)['SHARE'].sum().rename(columns={'SHARE':'Diurnal'})
diurnal_period_totals

,period,Diurnal
0,AM,0.211
1,MD,0.348
2,OV,0.133
3,PM,0.307


### Summarize Converted Vehicle Roster

In [9]:
start_time = time.time()

In [10]:
def expand_data(df):
    """
    attach text labels to the data using Categories
    
    """
    VTYPE_LABEL = {
        1:  'REG_SOV', 
        2:  'REG_HOV',
        3:  'REG_Serv',
        4:  'REG_Cargo',
        5:  'CAV_SOV',
        6:  'CAV_HOV',
        7:  'CAV_Serv',
        8:  'CAV_Cargo',
        9:  'TNC',
        
    }
    
    CAV_LABEL = {
        1:  'REG',
        2:  'REG',
        3:  'REG',
        4:  'REG',
        5:  'CAV',
        6:  'CAV',
        7:  'CAV',
        8:  'CAV',
        9:  'REG',
    }

    
    PURP_LABEL = {
        1: 'WK',
        10: 'WK',
        2: 'NW',
        20: 'NW',
        3: 'Taxi',
        30: 'Taxi',
        4: 'TNC',
        40: 'TNC',
        5: 'Serv',
        50: 'Serv',
        6: 'Cargo',
        60: 'Cargo',
        
    }
    
    def period(r):
        r = r / 60.0
        if 0 <= r < 6 or 19 <= r < 24:
            return 'OV'
        elif 6 <= r < 9:
            return 'AM'
        elif 9 <= r < 15:
            return 'MD'
        else:
            return 'PM' 
    
    columns = df.columns.tolist()
    if 'vehtype' in columns:
        df['vehtype2'] = df.vehtype.map(VTYPE_LABEL)
        df['cav'] = df.vehtype.map(CAV_LABEL)
        df['vehtype2'] = df['vehtype2'].astype("category")
        df['cav'] = df['cav'].astype("category")
    if 'TP' in columns:
        df['TP2'] = df.TP.map(PURP_LABEL).astype("category")
    
    if 'stime' in columns:
        df['period'] = df.stime.apply(period).astype("category")
    
    df['ev'] = np.where(df.TP >= 10, 'EV', 'NonEV')
    df['ev'] = df.ev.astype('category')
    

In [11]:
def read_flat_trip_file(flat_trip_file, chunksize=5_000_000):
#     if not write_feather:
#         if feather_trip_file:
#             if not os.path.isfile(feather_trip_file):
#                 print('Error: Feather Trip File not Found {:s}'.format(feather_trip_file))
#                 return
#             else:
#                 df = pd.read_feather(feather_trip_file)
#                 return df

    if not os.path.isfile(flat_trip_file):
        print('Error: Trip File not Found {:s}'.format(flat_trip_file))
        return
    
    # write the feather file in the same directory
#     feather_trip_path, filename = os.path.dirname(flat_trip_file), os.path.splitext(os.path.basename(flat_trip_file))[0] + '.feather'
#     feather_trip_file = os.path.join(feather_trip_path, filename)
    
    dtypes = {
        'vid': np.int32, 
        'usec': np.int32,
        'dsec': np.int32,
        'stime': np.float16,
        'vehcls': np.uint8,
        'vehtype': np.uint8,
        'ioc': np.uint8,
        '#ONode': np.int32,
        '#IntDe': np.uint8,
        'info': np.uint8,
        'ribf': np.float16,
        'comp': np.float16,
        'Izone': np.uint16,
        'Evac': np.uint8,
        'InitPos': np.float32,
        'VoT': np.float16,
        'tFlag': np.uint8,
        'pArrTime': np.float16,
        'TP': np.uint8,
        'IniGas': np.float16,
        'DZone': np.uint16,
        'waitTime': np.float16
    }
    
    usecols = ('vid', 'usec', 'dsec', 'stime', 'vehtype', 'ioc', 'Izone', 'VoT', 'TP', 'DZone')
    
    df_list = []
    
    for df_chunk in tqdm(pd.read_csv(flat_trip_file, usecols=usecols, dtype=dtypes, chunksize=chunksize)):
    
        expand_data(df_chunk)
        df_list.append(df_chunk) 
    
    df = pd.concat(df_list)
    del df_list
    gc.collect()
#     if write_feather:
#         df.to_feather(feather_trip_file)
#         print('Feather file written to {:s}'.format(feather_trip_file))
    return df
    

In [40]:
df_flat_trip_file = read_flat_trip_file(flat_trip_file)

A Jupyter Widget

In [41]:
print('Total Trips for DynusT = {:>,d}'.format(df_flat_trip_file.shape[0]))

Total Trips for DynusT = 19,755,937


In [42]:
# df_flat_trip_file.info()

In [43]:
# df_flat_trip_file.head()

#### <font color='blue'>Summary by all dimensions</font>

In [44]:
summary_roster_gb = df_flat_trip_file.groupby(['period', 'TP2', 'vehtype2', 'cav', 'ev'], as_index=False)['vid'].count().rename(columns={'vid':'trips', 'TP2':'purp'})
summary_roster_gb = summary_roster_gb[~summary_roster_gb.trips.isnull()]

#### <font color='blue'>Summary by Vehicle Type</font>

In [45]:
# summary = pd.DataFrame(df_flat_trip_file.vehtype2.value_counts(sort=False))
# summary = summary.style.format("{:,}")
# summary

### Summarize OMX Trip Tables

In [12]:
def parse_matrix_table(matrix_file_name, matrix_name, vots):
    """
    Parse the matrix name "amnhoai45" for vehicle type, value of time, period, and purpose
    :param matrix_name:
    :type str
    :param vots:        a dict for vots
    :return: a tuple, vehicle_type, occupancy, purpose, value_of_time, period
    """

    matrix_name = matrix_name.lower()

    vtype = occ = purp = vot = 0
    time_period = (0, 0)
    period = ''

    cav, ev = False, False

    if matrix_name.lower().find("da") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 5
            occ = 0
            cav = True
        elif matrix_name.lower().find("reg") >= 0:
            vtype = 1
            occ = 0
            cav = False

    elif matrix_name.lower().find("s2") >= 0 or matrix_name.find("a2") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 6
            occ = 1
            cav = True
        elif matrix_name.lower().find("reg") >= 0:
            vtype = 2
            occ = 1
            cav = False
    elif matrix_name.lower().find("s3") >= 0 or matrix_name.find("a3") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 6
            occ = 1
            cav = True
        elif matrix_name.lower().find("reg") >= 0:
            vtype = 2
            occ = 1
            cav = False
    elif matrix_name.lower().find("cargo") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 8
            occ = 2
            cav = True
        else:
            vtype = 4
            occ = 2
            cav = False
    elif matrix_name.lower().find("serv") >= 0:
        if matrix_name.lower().find("cav") >= 0:
            vtype = 7
            occ = 2
            cav = True
        else:
            vtype = 3
            occ = 2
            cav = False
    elif matrix_name.lower().find("taxi") >= 0:
        vtype = 2
        occ = 1
    elif matrix_name.lower().find("exta") >= 0:
        vtype = 2
        occ = 0
    elif matrix_name.lower().find("tnc") >= 0:
        vtype = 9
        occ = 1

    if matrix_name.find("_w") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 1
        else:
            purp = 10
            ev = True
    elif matrix_name.find("_nw") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 2
        else:
            purp = 20
            ev = True
    elif matrix_name.lower().find("taxi") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 3
        else:
            purp = 30
            ev = True
    elif matrix_name.lower().find("tnc") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 4
        else:
            purp = 40
            ev = True
    elif matrix_name.lower().find("serv") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 5
        else:
            purp = 50
            ev = True
    elif matrix_name.lower().find("cargo") >= 0:
        if matrix_file_name.lower().find("dynust nonev") >= 0:
            purp = 6
        else:
            purp = 60
            ev = True

    # Time periods
    if matrix_file_name.lower().find("am") >= 0:
        period = 'AM'
        time_period = '6..9'
    elif matrix_file_name.lower().find("md") >= 0:
        period = 'MD'
        time_period = '9..15'
    elif matrix_file_name.lower().find("pm") >= 0:
        period = 'PM'
        time_period = '15..19'
    else:
        period = 'OV'
        time_period = '0..6, 19..24'

    for key in vots.keys():
        if matrix_name.find(key) >= 0:
            vot = vots[key]
            break

    return dict(vtype=vtype, occ=occ, purp=purp, vot=vot, time_period=time_period, period=period, cav=cav, ev=ev)

In [13]:
def summarize_trip_tables(matrix_files, vots):
    summary_records = []
    for m in matrix_files:
#         print('Processing Matrix File {:s}'.format(m))
        h5 = h5py.File(m, 'r')
        group = '/'+list(h5.keys())[0]+'/'
        tables = h5[group].keys()
        for table in tqdm(tables, desc=os.path.split(m)[1].split('.')[0]):
            if table.find('all') >= 0:
                continue
            info = parse_matrix_table(m, table, vots)
            vtype = info['vtype']
            occ = info['occ']
            purp = info['purp']
            vot = info['vot']
            period = info['period']
            cav = info['cav']
            ev = info['ev']
            
            if cav:
                cav = 'CAV'
            else:
                cav = 'REG'
            
            if ev:
                ev = 'EV'
            else:
                ev = 'NonEV'
            
            od = h5[group + table][:]
            trips = od.sum()
            summary_records.append((table.lower(), period, purp, vot, vtype, occ, cav, ev, trips))
        h5.close()
    return pd.DataFrame(data=summary_records, columns=['table', 'period', 'purp', 'vot', 'vtype', 'occ', 'cav', 'ev', 'trips'])
    

In [15]:
summary_omx = summarize_trip_tables(matrix_files, vots)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [16]:
summary_omx['purp'] = summary_omx.purp.map(map_purp)
summary_omx['vehtype2'] = summary_omx.vtype.map(VTYPE_LABEL)

In [17]:
summary_omx_gb = summary_omx.groupby(['period', 'purp', 'vehtype2', 'cav', 'ev'], as_index=False)['trips'].sum()
summary_omx_gb = summary_omx_gb[summary_omx_gb.trips > 0]

In [18]:
summary_omx_gb

,period,purp,vehtype2,cav,ev,trips
0,AM,Cargo,REG_Cargo,REG,EV,1080.628
1,AM,Cargo,REG_Cargo,REG,NonEV,27153.224
2,AM,NW,CAV_HOV,CAV,EV,15266.016
3,AM,NW,CAV_HOV,CAV,NonEV,49620.449
4,AM,NW,CAV_SOV,CAV,EV,45523.698
5,AM,NW,CAV_SOV,CAV,NonEV,148003.935
6,AM,NW,REG_HOV,REG,EV,43610.020
7,AM,NW,REG_HOV,REG,NonEV,142030.603
8,AM,NW,REG_SOV,REG,EV,127373.622
9,AM,NW,REG_SOV,REG,NonEV,414497.098


#### Compare Vehicle Roster to OMX

In [58]:
join_cols = ['period', 'purp', 'vehtype2', 'cav', 'ev']
summary_comparison = pd.merge(left=summary_omx_gb, right=summary_roster_gb, on=join_cols, suffixes=('_OMX', '_DY'),)
summary_comparison['Abs_Diff'] = summary_comparison.trips_DY - summary_comparison.trips_OMX
summary_comparison['Rel_Diff'] = summary_comparison.Abs_Diff / summary_comparison.trips_OMX * 100.0
summary_comparison_show = summary_comparison.style.format({'trips_OMX': "{:,.2f}", 'trips_DY': '{:,.0f}', 'Rel_Diff': '{:.2f}%'})
summary_comparison_show

,period,purp,vehtype2,cav,ev,trips_OMX,trips_DY,Abs_Diff,Rel_Diff
0,AM,Cargo,REG_Cargo,REG,EV,73.24,73,-0.239968,-0.33%
1,AM,Cargo,REG_Cargo,REG,NonEV,"24,515.62","24,523",7.37997,0.03%
2,AM,NW,REG_HOV,REG,EV,"6,972.63","6,970",-2.62854,-0.04%
3,AM,NW,REG_HOV,REG,NonEV,"351,433.28","351,521",87.7152,0.02%
4,AM,NW,REG_SOV,REG,EV,"20,074.95","20,072",-2.94943,-0.01%
5,AM,NW,REG_SOV,REG,NonEV,"1,013,146.87","1,013,464",317.131,0.03%
6,AM,Serv,REG_Serv,REG,EV,"1,725.05","1,724",-1.0539,-0.06%
7,AM,Serv,REG_Serv,REG,NonEV,"97,526.96","97,579",52.0426,0.05%
8,AM,TNC,TNC,REG,EV,"8,519.67","8,521",1.3337,0.02%
9,AM,TNC,TNC,REG,NonEV,"449,645.08","449,671",25.9223,0.01%


In [59]:
total_OMX, total_DY = sum(summary_comparison.trips_OMX), sum(summary_comparison.trips_DY)
print('Total Trips for OMX    = {:>,.0f}'.format(total_OMX))
print('Total Trips for DynusT = {:>,.0f}'.format(total_DY))
print('Total Abs. Difference  = {:>,.0f}'.format(total_DY-total_OMX))
print('Total Rel. Difference  = {:>.2f}%'.format((total_DY-total_OMX)/total_OMX*100.0))

Total Trips for OMX    = 19,755,937
Total Trips for DynusT = 19,755,937
Total Abs. Difference  = -0
Total Rel. Difference  = -0.00%


#### Period Total Comparison

In [63]:
summary_comparison_period = summary_comparison.groupby('period', as_index=False)['trips_OMX', 'trips_DY'].sum()
summary_comparison_period['trips_OMX'] = summary_comparison_period.trips_OMX / sum(summary_comparison_period.trips_OMX)
summary_comparison_period['trips_DY'] = summary_comparison_period.trips_DY / sum(summary_comparison_period.trips_DY)
summary_comparison_period = pd.merge(summary_comparison_period, diurnal_period_totals, left_on='period', right_on='period')
summary_comparison_period

,period,trips_OMX,trips_DY,Diurnal
0,AM,0.159,0.159,0.211
1,MD,0.348,0.348,0.348
2,OV,0.171,0.172,0.133
3,PM,0.321,0.321,0.307


In [61]:
diurnal_period_totals

,Period,Diurnal
0,AM,0.211
1,MD,0.348
2,OV,0.133
3,PM,0.307


In [ ]:
summary_comparison.to_csv(os.path.join(os.path.dirname(flat_trip_file), 'summary_comparison.csv'), index=False)

In [ ]:
end_time = time.time()
execution_time = (end_time - start_time) / 60.0
print("ALL DONE in {:.0f} minutes".format(execution_time))

In [ ]:
# import itertools
# periods = ('AM', 'MD', 'PM', 'OV')
# purposes = ('WK', 'NW', 'Cargo', 'Serv', 'Taxi', 'TNC')
# CAV = ('REG', 'CAV')
# EV = ('EV', 'NonEV')
# pd.DataFrame(list(itertools.product(periods, purposes, CAV, EV))).to_csv(os.path.join(os.path.dirname(flat_trip_file), 'cat.csv'))

#### Old Format Matrices

In [64]:
matrix_folder = r'L:\DCS\Projects\_Legacy\60563434_SWIFT\400_Technical\410 Transportation Modeling\DATA\H-GAC\Trip Tables\OD\Y2017_OD_TOD'
matrices_am = ["OD AM3HR HBNW Vehicles", "OD AM3HR HBW Vehicles", "OD AM3HR NHB Vehicles", "OD AM3HR Other VEHICLEs"]
matrices_md = ["OD MD6HR HBNW Vehicles", "OD MD6HR HBW Vehicles", "OD MD6HR NHB Vehicles", "OD MD6HR Other VEHICLEs"]
matrices_pm = ["OD PM4HR HBNW Vehicles", "OD PM4HR HBW Vehicles", "OD PM4HR NHB Vehicles", "OD PM4HR Other VEHICLEs"]
matrices_ov = ["OD OV8HR HBNW Vehicles", "OD OV8HR HBW Vehicles", "OD OV8HR NHB Vehicles", "OD OV8HR Other VEHICLEs"]
matrices = matrices_am + matrices_md + matrices_pm + matrices_ov
matrix_files = [os.path.join(matrix_folder, m+'.omx') for m in matrices]

In [67]:
def summarize_trip_tables_oldformat(matrix_files):
    summary_records = []
    for m in matrix_files:
        h5 = h5py.File(m, 'r')
        period = None
        if m.lower().find("am") >= 0:
            period = 'AM'
        elif m.lower().find("md") >= 0:
            period = 'MD'
        elif m.lower().find("pm") >= 0:
            period = 'PM'
        else:
            period = 'OV'
        
        group = '/'+list(h5.keys())[0]+'/'
        tables = h5[group].keys()
        for table in tqdm(tables, desc=os.path.split(m)[1].split('.')[0]):
            if table.find('all') >= 0:
                continue
    
            purp = None
            if table.lower().find("hbw") >= 0:
                purp = 'HBW'
            elif table.lower().find("hnw") >= 0:
                purp = 'HNW'
            elif table.lower().find("nho") >= 0:
                purp = 'NHO'
            elif table.lower().find("nhw") >= 0:
                purp = 'NHW'
            else:
                purp = 'MISC'
            
            od = h5[group + table][:]
            trips = od.sum()
            summary_records.append((table.lower(), period, purp, trips))
        h5.close()
    return pd.DataFrame(data=summary_records, columns=['table', 'period', 'purp', 'trips'])

In [68]:
summary_omx_oldformat = summarize_trip_tables_oldformat(matrix_files)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [71]:
summary_omx_oldformat.groupby('period', as_index=False)['trips'].sum()

,period,trips
0,AM,3341684.091
1,MD,6630650.158
2,OV,3202482.509
3,PM,6128534.540


In [22]:
def summarize_trip_tables_customize(matrix_files, vots):
    summary_records = []
    for m in matrix_files:
#         print('Processing Matrix File {:s}'.format(m))
        h5 = h5py.File(m, 'r')
        group = '/'+list(h5.keys())[0]+'/'
        tables = h5[group].keys()
        for table in tqdm(tables, desc=os.path.split(m)[1].split('.')[0]):
            if table.find('all') >= 0:
                continue
            info = parse_matrix_table(m, table, vots)
            vtype = info['vtype']
            occ = info['occ']
            purp = info['purp']
            vot = info['vot']
            period = info['period']
            cav = info['cav']
            ev = info['ev']
            
            if cav:
                cav = 'CAV'
            else:
                cav = 'REG'
            
            if ev:
                ev = 'EV'
            else:
                ev = 'NonEV'
            
            select_dzone = 1919
            od = h5[group + table][:]
            trips = od[:,select_dzone-1].sum()
            summary_records.append((table.lower(), period, purp, vot, vtype, occ, cav, ev, trips))
        h5.close()
    return pd.DataFrame(data=summary_records, columns=['table', 'period', 'purp', 'vot', 'vtype', 'occ', 'cav', 'ev', 'trips'])

In [23]:
summary_omx_customize = summarize_trip_tables_customize(matrix_files, vots)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [21]:
summary_omx_customize.loc[summary_omx_customize.purp.isin((1,2,10,20)), 'trips'].sum()

34.7398

In [20]:
summary_omx_customize.head()

,table,period,purp,vot,vtype,occ,cav,ev,trips
0,reg_wi3a2,AM,10,35.840,2,1,REG,EV,0.000
1,reg_wi2da,AM,10,15.040,1,0,REG,EV,0.048
2,reg_wi4da,AM,10,27.520,1,0,REG,EV,0.153
3,cav_wi5da,AM,10,37.120,5,0,CAV,EV,0.084
4,cargo,AM,60,0.000,4,2,REG,EV,0.077


#### Scratch

In [20]:
summary_omx_gb.loc[summary_omx_gb.purp.isin(('TNC', 'Taxi'))].groupby('period', as_index=False)['trips'].sum()

,period,trips
0,AM,3149419.081
1,MD,8638338.998
2,OV,3632567.064
3,PM,7275598.955


In [21]:
summary_omx_gb.loc[summary_omx_gb.purp.isin(('TNC', 'Taxi'))]

,period,purp,vehtype2,cav,ev,trips
12,AM,TNC,TNC,REG,EV,706540.865
13,AM,TNC,TNC,REG,NonEV,2331421.278
14,AM,Taxi,REG_HOV,REG,EV,5693.061
15,AM,Taxi,REG_HOV,REG,NonEV,105763.877
36,MD,TNC,TNC,REG,EV,1921311.210
37,MD,TNC,TNC,REG,NonEV,6502689.043
38,MD,Taxi,REG_HOV,REG,EV,9411.984
39,MD,Taxi,REG_HOV,REG,NonEV,204926.762
60,OV,TNC,TNC,REG,EV,804617.869
61,OV,TNC,TNC,REG,NonEV,2697243.725
